# Specfem2D Local Example 

To demonstrate the inversion capabilities of SeisFlows3, we will run a 2D synthetic-synthetic example on our local machine (Linux workstation running CentOS 7) using [SPECFEM2D](https://geodynamics.org/cig/software/specfem2d/). 

Many of the setup steps here will likely be unique to our OS and workstation (e.g., compiling SPECFEM2D), however they are intedned to serve as templates for new Users wanting to explore SeisFlows3. 

We'll also be working in our `seisflows3` Conda environment, see !!! DOCS PAGE !!! for how to install and activate the required Conda environment.

The steps to take are as follows:

1. Generate and populate SeisFlows3 parameters.yaml file
2. Initate SeisFlows3 working directory
3. Run an inversion up to the forward simulations, explore results
4. Complete inversion up to model update, explore results

In [1]:
import os
import glob
import shutil

## 1. Setup SPECFEM2D 
### a. Download and compile codebase

First we'll download and compile SPECFEM2D to generate the binaries necessary to run our simulations. We will then populate a new SPECFEM2D working directory that will be used by SeisFlows3. We'll use to Python OS module to do our filesystem processes just to keep everything in Python, but this can easily be accomplished in bash.

In [2]:
# !!! If following along, please edit this path to point to your own working directory here 
# !!! change this to wherever you are running this example. The remainder of the paths will
# !!! adjust based on the value of WORKDIR
WORKDIR = "/home/bchow/Work/work/sf3_specfem2d_example" 

# The SPECFEM2D repository that we will download from GitHub
SPECFEM2D_ORIGINAL = os.path.join(WORKDIR, "specfem2d")
SPECFEM2D_BIN_ORIGINAL = os.path.join(SPECFEM2D_ORIGINAL, "bin")
SPECFEM2D_DATA_ORIGINAL = os.path.join(SPECFEM2D_ORIGINAL, "DATA")
TAPE_2007_EXAMPLE = os.path.join(SPECFEM2D_ORIGINAL, "EXAMPLES", "Tape2007")

# The SPECFEM2D working directory that we will create using the downloaded repo
SPECFEM2D_WORKDIR = os.path.join(WORKDIR, "specfem2d_workdir")
SPECFEM2D_BIN = os.path.join(SPECFEM2D_WORKDIR, "bin")
SPECFEM2D_DATA = os.path.join(SPECFEM2D_WORKDIR, "DATA")
SPECFEM2D_OUTPUT = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES")

# These are locations of velocity models we will generate using SPECFEM2D
SPECFEM2D_MODEL_INIT = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES_INIT")
SPECFEM2D_MODEL_TRUE = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES_TRUE")

In [10]:
# Download SPECFEM2D from GitHub, devel branch for latest codebase
os.chdir(WORKDIR)
if not os.path.exists("specfem2d"):
    !git clone --recursive --branch devel https://github.com/geodynamics/specfem2d.git

Cloning into 'specfem2d'...
Updating files: 100% (2266/2266), done.
Submodule 'm4' (https://github.com/geodynamics/autoconf_cig.git) registered for path 'm4'
Cloning into '/home/bchow/Work/work/sf3_specfem2d_example/specfem2d/m4'...


Submodule path 'm4': checked out 'f2b2f3b21db9175aa3de73031b8c22e711065848'


In [10]:
# Run the compilation step which sets up the Makefile
os.chdir(SPECFEM2D_ORIGINAL)
if not os.path.exists("./config.log"):
    os.system("./configure")

## ---------------------------- ##
## setting up compilation flags ##
## ---------------------------- ##
checking build system type... x86_64-unknown-linux-gnu
checking host system type... x86_64-unknown-linux-gnu
checking for gfortran... gfortran
checking whether the Fortran compiler works... yes
checking for Fortran compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU Fortran compiler... yes
checking whether gfortran accepts -g... yes
configure: running /bin/sh ./flags.guess
checking how to get verbose linking output from gfortran... -v
checking for Fortran libraries of gfortran...  -L/usr/lib/gcc/x86_64-redhat-linux/4.8.5 -L/usr/lib/gcc/x86_64-redhat-linux/4.8.5/../../../../lib64 -L/lib/../lib64 -L/usr/lib/../lib64 -L/usr/lib/gcc/x86_64-redhat-linux/4.8.5/../../.. -lgfortran -lm -lquadmath
checking for gcc... gcc
checking whether w

fatal: No names found, cannot describe anything.


config.status: creating Makefile
config.status: creating setup/constants.h
config.status: creating setup/constants_tomography.h
config.status: creating setup/precision.h
config.status: creating setup/config.fh
config.status: creating DATA/Par_file
config.status: creating DATA/SOURCE
config.status: creating setup/config.h
config.status: setup/config.h is unchanged
config.status: executing bin commands
config.status: executing obj commands
config.status: executing DATA commands
config.status: executing OUTPUT_FILES commands

## ---------------- ##
## Specfem 2D 7.0.0 ##
## ---------------- ##

./configure has completed and set up a default configuration to build.

You may wish to modify the following files before running a simulation:
  DATA/Par_file           Set parameters affecting the simulation.
  DATA/SOURCE             Set the source parameters before running the solver.



In [12]:
# Generate the SPECFEM2D binaries, the numerical solver tools that SeisFlows3 will automate
if not os.path.exists("bin"):
    os.system("make all")

gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/shared_par.shared_module.o src/shared/shared_par.F90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/meshfem2D_par.mesh_module.o src/meshfem2D/meshfem2D_par.f90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/compute_ele

fatal: No names found, cannot describe anything.


gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/meshfem2D.mesh.o src/meshfem2D/meshfem2D.F90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/exit_mpi.shared.o src/shared/exit_mpi.F90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/parallel.sharedmpi.o src/shared/paral

gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/check_stability.spec.o src/specfem2D/check_stability.F90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/comp_source_time_function.spec.o src/specfem2D/comp_source_time_function.f90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup 

gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/compute_forces_poro_solid.spec.o src/specfem2D/compute_forces_poro_solid.f90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/compute_forces_poro_viscous_damping.spec.o src/specfem2D/compute_forces_poro_viscous_damping.f90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-fu

...

gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/pml_compute_memory_variables.spec.o src/specfem2D/pml_compute_memory_variables.f90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/prepare_absorb.spec.o src/specfem2D/prepare_absorb.f90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./se

gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/setup_sources_receivers.spec.o src/specfem2D/setup_sources_receivers.F90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/sort_array_coordinates.spec.o src/specfem2D/sort_array_coordinates.F90
gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  

gcc -c -g -O2  -I./setup -I./external_libs/libjpeg -o obj/jddctmgr.cc_jpeg.o external_libs/libjpeg/jddctmgr.c
gcc -c -g -O2  -I./setup -I./external_libs/libjpeg -o obj/jdhuff.cc_jpeg.o external_libs/libjpeg/jdhuff.c
gcc -c -g -O2  -I./setup -I./external_libs/libjpeg -o obj/jdinput.cc_jpeg.o external_libs/libjpeg/jdinput.c
gcc -c -g -O2  -I./setup -I./external_libs/libjpeg -o obj/jdmainct.cc_jpeg.o external_libs/libjpeg/jdmainct.c
gcc -c -g -O2  -I./setup -I./external_libs/libjpeg -o obj/jdmarker.cc_jpeg.o external_libs/libjpeg/jdmarker.c
gcc -c -g -O2  -I./setup -I./external_libs/libjpeg -o obj/jdmaster.cc_jpeg.o external_libs/libjpeg/jdmaster.c
gcc -c -g -O2  -I./setup -I./external_libs/libjpeg -o obj/jdmerge.cc_jpeg.o external_libs/libjpeg/jdmerge.c
gcc -c -g -O2  -I./setup -I./external_libs/libjpeg -o obj/jdpostct.cc_jpeg.o external_libs/libjpeg/jdpostct.c
gcc -c -g -O2  -I./setup -I./external_libs/libjpeg -o obj/jdsample.cc_jpeg.o external_libs/libjpeg/jdsample.c
gcc -c -g -O2  -I.


gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/adj_seismogram.aux.o src/auxiliaries/adj_seismogram.f90

building xadj_seismogram

gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -o bin/xadj_seismogram obj/adj_seismogram.aux.o

gfortran -g -O2 -std=f2008 -fimplicit-none -fmax-errors=10 -pedantic -pedantic-errors -Waliasing -Wampersand -Wcharacter-truncation -Wline-truncation -Wsurprising -Wno-tabs -Wunderflow -ffpe-trap=invalid,zero,overflow -Wunused -O3 -finline-functions  -J./obj -I./obj -I.  -I./setup -c -o obj/check_quality_external_mesh.aux.

0

In [11]:
# Check out the binary files that have been created
!ls bin/

xadj_seismogram		      xconvolve_source_timefunction  xspecfem2D
xcheck_quality_external_mesh  xmeshfem2D		     xsum_kernels
xcombine_sem		      xsmooth_sem


### b. Create a separate SPECFEM2D working directory

Next we'll create a separate SPECFEM2D working directory. The intent here is to isolate the original SPECFEM2D repository to protect it from user errors, such as accidentally deleting files while performing file manipulations. This is not a mandatory step, but helps keep things cleaner in the long run.

In [16]:
# Let's keep track of all the original directories. To run, SPECFEM only requires bin/ and DATA/ directories
print("\n".join([SPECFEM2D_ORIGINAL, SPECFEM2D_BIN_ORIGINAL, SPECFEM2D_DATA_ORIGINAL]))

/home/bchow/Work/work/sf3_specfem2d_example/specfem2d
/home/bchow/Work/work/sf3_specfem2d_example/specfem2d/bin
/home/bchow/Work/work/sf3_specfem2d_example/specfem2d/DATA


In [36]:
# Incase we've run this docs page before, delete the working directory before remaking
if os.path.exists(SPECFEM2D_WORKDIR):
    shutil.rmtree(SPECFEM2D_WORKDIR)

os.mkdir(SPECFEM2D_WORKDIR)
os.chdir(SPECFEM2D_WORKDIR)

# Copy the binary files incase we update the source code. These can also be symlinked.
shutil.copytree(SPECFEM2D_BIN_ORIGINAL, "bin")

# Copy the DATA/ directory because we will be making edits here frequently and it's useful to
# retain the original files for reference. We will be running one of the example problems: Tape2007
shutil.copytree(os.path.join(TAPE_2007_EXAMPLE, "DATA"), "DATA")

'DATA'

In [38]:
# As a check that this docs page can still run, we will go ahead and run the Tape2007 example
os.chdir(TAPE_2007_EXAMPLE)
!ls
!./run_this_example.sh

DATA			    run_this_example.sh
README			    wavefield_Tape2007_132rec_checker.pdf
REF_SEIS		    wavefield_Tape2007_onerec_homo.pdf
run_this_example_onerec.sh
running example: Tue Mar  1 12:04:05 AKST 2022

(will take about 1 minute)


   setting up example...


running mesher...


 **********************************************
 *** Specfem 2-D Mesher - serial version    ***
 **********************************************

 Running Git version of the code corresponding to commit cf89366717d9435985ba852ef1d41a10cee97884
 dating From Date:   Mon Nov 29 23:20:51 2021 -0800

 Reading the parameter file...

 Title of the simulation: Tape-Liu-Tromp (GJI 2007)


 Receiver lines:
   Nb of line sets =          132

   using existing STATIONS file: ./DATA/STATIONS
   file name is ./DATA/STATIONS
   found          132  receivers

 Materials:
   Nb of solid, fluid or porous materials =            1

 --------
 Material #           1  isotropic
 rho,cp,cs   =    2600.0000000000000        5800.00000

   done reading external model

   setting up material arrays
   setting up anisotropic arrays
   all material arrays done

 Domains:
   total number of acoustic elements        =            0
   total number of elastic elements         =         1600
   total number of poroelastic elements     =            0

 Setting up MPI communication arrays

   maximum number of MPI interfaces (for a single slice) =            0

   This is a single process simulation, no need for MPI communication

   number of outer elements  =            0
   number of inner elements  =         1600

   percentage of outer elements    0.00000000     %
   percentage of inner elements    100.000000     %

   number of global nodes in outer elements  =            0
   number of global nodes in inner elements  =        25921

   determining communication phases:
   elastic domains:
   total number of outer/inner elements =            0        1600
   total percentage of outer elements    0.00000000     %
   total 

 Color image maximum amplitude =    0.0000000000000000     
 Color image created

 Creating color image of size          763  x          800  for time step          200
 drawing scalar image of the forward wavefield displacement...
 Color image maximum amplitude =    0.0000000000000000     
 Color image created


 ******************************************************************
Time step number     400   t =   23.9400 s out of    4800
 ******************************************************************
 We have done    8.31423187     % of the total
 Max norm of vector field in solid (elastic) =    1.62307188E-18
 Elapsed time in seconds =   0.37600016593933105     
 Elapsed time in hh:mm:ss =      0 h 00 m 00 s
 Mean elapsed time per time step in seconds =    9.4000041484832765E-004
 Time steps remaining =         4400
 Estimated remaining time in seconds =    4.1360018253326416     
 Estimated remaining time in hh:mm:ss =      0 h 00 m 04 s
 Estimated total run time in seconds =    4

 Color image maximum amplitude =    1.3288295827805996E-002
 Color image created


 ******************************************************************
Time step number    2800   t =  167.9400 s out of    4800
 ******************************************************************
 We have done    58.3246498     % of the total
 Max norm of vector field in solid (elastic) =    3.41156055E-03
 Elapsed time in seconds =    5.1050000190734863     
 Elapsed time in hh:mm:ss =      0 h 00 m 05 s
 Mean elapsed time per time step in seconds =    1.8232142925262451E-003
 Time steps remaining =         2000
 Estimated remaining time in seconds =    3.6464285850524902     
 Estimated remaining time in hh:mm:ss =      0 h 00 m 03 s
 Estimated total run time in seconds =    8.7514286041259766     
 Estimated total run time in hh:mm:ss =      0 h 00 m 08 s
 The run will finish approximately on: Tue Mar 01, 2022 12:04

 Creating color image of size          763  x          800  for time step         2800


0

In [40]:
# Quick check that this example problem runs
assert(os.path.exists("OUTPUT_FILES/AA.S000000.BXY.semd")), "Example did not run, the remainder the doc is likely not to work"

### c. Setup the SeisFlows3 SPECFEM2D working directory

Now we need to manually set up our SPECFEM2D working directory. The necessary elements of this working directory are

1. bin/ directory containing SPECFEM binaries
2. DATA/ directory containing SOURCE and STATION files, as well as a SPECFEM Par_file
3. Database.bin files which define the starting (and target) models

We will just do a single source-receiver run to keep things as simple as possible.

In [49]:
# First we will set the correct SOURCE and STATION files, this is the same task as shown in 
# ./run_this_example.sh
os.chdir(SPECFEM2D_DATA)

# Symlink source 001 as our main source
if os.path.exists("SOURCE"):
    os.remove("SOURCE")
os.symlink("SOURCE_001", "SOURCE")

# Symlink the correct Par_file
if os.path.exists("Par_file"):
    os.remove("Par_file")
os.symlink("Par_file_Tape2007_onerec", "Par_file")

!ls

interfaces_Tape2007.dat		     SOURCE_003  SOURCE_012  SOURCE_021
model_velocity.dat_checker	     SOURCE_004  SOURCE_013  SOURCE_022
Par_file			     SOURCE_005  SOURCE_014  SOURCE_023
Par_file_Tape2007_132rec_checker     SOURCE_006  SOURCE_015  SOURCE_024
Par_file_Tape2007_onerec	     SOURCE_007  SOURCE_016  SOURCE_025
proc000000_model_velocity.dat_input  SOURCE_008  SOURCE_017  STATIONS
SOURCE				     SOURCE_009  SOURCE_018  STATIONS_checker
SOURCE_001			     SOURCE_010  SOURCE_019
SOURCE_002			     SOURCE_011  SOURCE_020


In [15]:
# SPECFEM requires that we create the OUTPUT_FILES directory before running
os.chdir(SPECFEM2D_WORKDIR)
if os.path.exists(SPECFEM2D_OUTPUT):
    shutil.rmtree(SPECFEM2D_OUTPUT)
os.mkdir(SPECFEM2D_OUTPUT)

### d. Generate initial and target models

Since we're doing a synthetic-synthetic inversion, we need to manually set up the velocity models with which we will generate our synthetic waveforms (initial model) and our 'data' (target model). Luckily the starting model is defined as a homogeneous halfspace by the example problem. We will generate our target model by slightly perturbing our initial model.

In [28]:
# Let's run the mesher to generate our initial model
os.chdir(SPECFEM2D_WORKDIR)
!./bin/xmeshfem2D

# Make sure we don't overwrite this initial model when creating our target model
os.rename("OUTPUT_FILES", "OUTPUT_FILES_INIT")


 **********************************************
 *** Specfem 2-D Mesher - serial version    ***
 **********************************************

 Running Git version of the code corresponding to commit cf89366717d9435985ba852ef1d41a10cee97884
 dating From Date:   Mon Nov 29 23:20:51 2021 -0800

 Reading the parameter file...

 Title of the simulation: Tape-Liu-Tromp (GJI 2007)


 Receiver lines:
   Nb of line sets =            1

 Materials:
   Nb of solid, fluid or porous materials =            1

 --------
 Material #           1  isotropic
 rho,cp,cs   =    2600.0000000000000        5800.0000000000000        3500.0000000000000     
 Qkappa, Qmu =    10.000000000000000        10.000000000000000     
 Material is solid
 --------


 Mesh from internal meshing:

 Reading interface data from file: ./DATA/interfaces_Tape2007.dat
 Reading            2  points for interface            1
 Reading            2  points for interface            2

 Total number of layers in z direction =      

In [44]:
# We can use the SeisFlows option 'sempar' to directly edit the SPECFEM Par_file in the command line
# We want to perturb the initial model which is defined with the Par_file, to create our target model.
# The subargument 'velocity_model' will let us view and edit the velocity model
os.chdir(SPECFEM2D_DATA)
! seisflows sempar velocity_model

# Lets edit the Par_file by increasing velocities ~10% 
! seisflows sempar velocity_model '1 1 2600.d0 5900.d0 3550.0d0 0 0 10.d0 10.d0 0 0 0 0 0 0'


1 1 2600.d0 5900.d0 3550.0d0 0 0 10.d0 10.d0 0 0 0 0 0 0



1 1 2600.d0 5900.d0 3550.0d0 0 0 10.d0 10.d0 0 0 0 0 0 0

->

1 1 2600.d0 5900.d0 3550.0d0 0 0 10.d0 10.d0 0 0 0 0 0 0


In [45]:
# And then we run the mesher again to get our target velocity model
os.chdir(SPECFEM2D_WORKDIR)
if os.path.exists(SPECFEM2D_OUTPUT):
    shutil.rmtree(SPECFEM2D_OUTPUT)
os.mkdir(SPECFEM2D_OUTPUT)

! ./bin/xmeshfem2D
os.rename("OUTPUT_FILES", "OUTPUT_FILES_TRUE")


 **********************************************
 *** Specfem 2-D Mesher - serial version    ***
 **********************************************

 Running Git version of the code corresponding to commit cf89366717d9435985ba852ef1d41a10cee97884
 dating From Date:   Mon Nov 29 23:20:51 2021 -0800

 Reading the parameter file...

 Title of the simulation: Tape-Liu-Tromp (GJI 2007)


 Receiver lines:
   Nb of line sets =            1

 Materials:
   Nb of solid, fluid or porous materials =            1

 --------
 Material #           1  isotropic
 rho,cp,cs   =    2600.0000000000000        5900.0000000000000        3550.0000000000000     
 Qkappa, Qmu =    10.000000000000000        10.000000000000000     
 Material is solid
 --------


 Mesh from internal meshing:

 Reading interface data from file: ./DATA/interfaces_Tape2007.dat
 Reading            2  points for interface            1
 Reading            2  points for interface            2

 Total number of layers in z direction =      

In [33]:
# Great, we have all the necessary SPECFEM files to run our SeisFlows3 inversion
!ls

bin  DATA  OUTPUT_FILES_INIT  OUTPUT_FILES_TRUE


## 2. Initiate SeisFlows3
### a. Create working directory and parameter file

Now that we've completed the SPECFEM2D setup, we can set up SeisFlows3. As with SPECFEM, SeisFlows3 requires a parameter file. We can use the built-in SeisFlows3 command line tools to generate and populate this parameter file.

In [47]:
# Let's create a SeisFlows3 parameter file in our working directory.
# We can figure out what command we need by runing the help command
os.chdir(WORKDIR)
!seisflows -h

usage: seisflows [-h] [-w [WORKDIR]] [-p [PARAMETER_FILE]]
                 [--path_file [PATH_FILE]]
                 {setup,modules,configure,init,submit,resume,restart,clean,par,sempar,check,convert,reset,inspect,debug,edit}
                 ...


                     SeisFlows: Waveform Inversion Package                      


optional arguments:
  -h, --help            show this help message and exit
  -w [WORKDIR], --workdir [WORKDIR]
                        The SeisFlows working directory, default: cwd
  -p [PARAMETER_FILE], --parameter_file [PARAMETER_FILE]
                        Parameters file, default: 'parameters.yaml'
  --path_file [PATH_FILE]
                        Legacy path file, default: 'paths.py'

command:
  Available SeisFlows arguments and their intended usages

    setup               Setup working directory from scratch
    modules             Print available module names
    configure           Fill parameter file with defaults
    init                Initia

In [53]:
# The command 'setup' is meant to setup a working directory from scratch. Lets do
! rm parameters.yaml
! ls
! echo
! seisflows setup
! ls

specfem2d  specfem2d_workdir

parameters.yaml  specfem2d  specfem2d_workdir


In [54]:
# This has created the parameters.yaml file, which controls the entire SeisFlows3 package
# Let's have a look at this file, which has not yet been populated
!cat parameters.yaml

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
#                 Seisflows YAML Parameter File and Path Input
#
#  For NoneType, set variables to `None` or `null`. For infinity, set to `inf`
#
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# These modules correspond to the structure of the source code, and determine
# SeisFlows' behavior at runtime. Check the source code directory for available 
# module names. Each module will require its own set of sub parameters. 
#
# To fill this parameter file with docstrings and default values, run:
#
# > seisflows configure
#
#                                    MODULES
#                                    -------
#
# WORKFLOW:    The method for running SeisFlows. Equivalent to main()
# SOLVER:      External numerical solver to use for waveform simulations.
# SYSTEM:      Computer architecture of the system being used to run SeisFlows
# OPTIMIZE:    Optimization algorithm fo

In [55]:
# We will need to edit the SOLVER module to let SeisFlows3 know that we're using the 2D code
! seisflows par solver specfem2d
! cat parameters.yaml


	SOLVER: specfem3d -> specfem2d

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
#                 Seisflows YAML Parameter File and Path Input
#
#  For NoneType, set variables to `None` or `null`. For infinity, set to `inf`
#
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# These modules correspond to the structure of the source code, and determine
# SeisFlows' behavior at runtime. Check the source code directory for available 
# module names. Each module will require its own set of sub parameters. 
#
# To fill this parameter file with docstrings and default values, run:
#
# > seisflows configure
#
#                                    MODULES
#                                    -------
#
# WORKFLOW:    The method for running SeisFlows. Equivalent to main()
# SOLVER:      External numerical solver to use for waveform simulations.
# SYSTEM:      Computer architecture of the system being used to run SeisFlows
# OPTI

In [56]:
# And now we can run the 'configure' command which will populate the parameter file
# based on the chosen modules
!seisflows configure

In [57]:
!cat parameters.yaml

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
#                 Seisflows YAML Parameter File and Path Input
#
#  For NoneType, set variables to `None` or `null`. For infinity, set to `inf`
#
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# These modules correspond to the structure of the source code, and determine
# SeisFlows' behavior at runtime. Check the source code directory for available 
# module names. Each module will require its own set of sub parameters. 
#
# To fill this parameter file with docstrings and default values, run:
#
# > seisflows configure
#
#                                    MODULES
#                                    -------
#
# WORKFLOW:    The method for running SeisFlows. Equivalent to main()
# SOLVER:      External numerical solver to use for waveform simulations.
# SYSTEM:      Computer architecture of the system being used to run SeisFlows
# OPTIMIZE:    Optimization algorithm fo

SeisFlows3 has attempted to fill in default parameter values where possible, but values that the User MUST set are denoted by the value:

!!! REQUIRED PARAMETER !!!

We will need to set all of these manually before running SeisFlows3. We'll go through these one at a time and leave comments next to the choice for each.

In [58]:
! seisflows par walltime 10  # master job run time in minutes
! seisflows par tasktime 1  # individual job run time in minutes
! seisflows par materials elastic  # how the velocity model is parameterized
! seisflows par density constant  # update density or keep constant
! seisflows par nt 5000  # set by SPECFEM2D Par_file
! seisflows par dt .006  # set by SPECFEM2D Par_file
! seisflows par f0 5.196  # set by SPECFEM2D Par_file
! seisflows par format ascii  # how to output synthetic seismograms
! seisflows par begin 1  # first iteration
! seisflows par end 1  # final iteration
! seisflows par case synthetic  # synthetic-synthetic means we need both INIT and TRUE models
! seisflows par misfit waveform  # direct waveform comparison for the misfit function

# Use Python syntax here to access path constants
os.system(f"seisflows par specfem_bin {SPECFEM2D_BIN}")  # set path to SPECFEM2D binaries
os.system(f"seisflows par specfem_data {SPECFEM2D_DATA}")  # set path to SEPCFEM2D DATA/
os.system(f"seisflows par model_init {SPECFEM2D_MODEL_INIT}")  # set path to INIT model
os.system(f"seisflows par model_true {SPECFEM2D_MODEL_TRUE}")  # set path to TRUE model


	WALLTIME: !!! REQUIRED PARAMETER !!! -> 10


	TASKTIME: !!! REQUIRED PARAMETER !!! -> 1


	MATERIALS: !!! REQUIRED PARAMETER !!! -> elastic


	DENSITY: !!! REQUIRED PARAMETER !!! -> constant


	NT: !!! REQUIRED PARAMETER !!! -> 5000


	DT: !!! REQUIRED PARAMETER !!! -> .006


	F0: !!! REQUIRED PARAMETER !!! -> 5.196


	FORMAT: !!! REQUIRED PARAMETER !!! -> ascii


	BEGIN: !!! REQUIRED PARAMETER !!! -> 1


	END: !!! REQUIRED PARAMETER !!! -> 1


	CASE: !!! REQUIRED PARAMETER !!! -> synthetic


	SPECFEM_BIN: !!! REQUIRED PATH !!! -> /home/bchow/Work/work/sf3_specfem2d_example/specfem2d_workdir/bin


	SPECFEM_DATA: !!! REQUIRED PATH !!! -> /home/bchow/Work/work/sf3_specfem2d_example/specfem2d_workdir/DATA


	MODEL_INIT: !!! REQUIRED PATH !!! -> /home/bchow/Work/work/sf3_specfem2d_example/specfem2d_workdir/OUTPUT_FILES_INIT


	MODEL_TRUE:  -> /home/bchow/Work/work/sf3_specfem2d_example/specfem2d_workdir/OUTPUT_FILES_TRUE



0

In [4]:
# We will also need to edit the SPECFEM2D Par_file to read our pre-built velocity
# model rather than the meshing parameters
os.chdir(SPECFEM2D_DATA)
! seisflows sempar model
! seisflows sempar model gll


	MODEL                           = default


	MODEL = default -> gll



### b. Generate SeisFlows3 working state

The SeisFlows3 command 'init' will generate the required SeisFlows3 working state, a collection of pickle and JSON files which define an active Python environment.This allows the user to explore the working state before running anything. This is not a required step but is useful for first-time users.

In [5]:
os.chdir(WORKDIR)

In [ ]:
! seisflows init

In [5]:
! ls output

seisflows_optimize.p	   seisflows_postprocess.p  seisflows_system.p
seisflows_parameters.json  seisflows_preprocess.p   seisflows_workflow.p
seisflows_paths.json	   seisflows_solver.p


In [6]:
! head output/seisflows_parameters.json

{
    "BACKPROJECT": null,
    "BEGIN": 1,
    "CASE": "synthetic",
    "COMPONENTS": "ZNE",
    "DENSITY": "constant",
    "DT": 0.006,
    "END": 1,
    "F0": 5.196,
    "FILTER": null,
